# 🧠 Final Project — Reported Crime (STARS Category Offenses) after 2024‑06‑03
**Course:** IT2053C • **Author:** Silas Curry

> Dataset: *Reported Crime (STARS Category Offenses) on or after 6/3/2024* — City of Cincinnati Open Data

### Executive Summary
- **Context / Stakeholder:** City of Cincinnati public safety stakeholders (CPD, city leadership, neighborhood councils) require timely, interpretable insights into crime patterns after CPD’s RMS transition on **June 3, 2024**.
- **Goal:** Summarize spatiotemporal patterns in reported incidents, identify high‑volume categories and neighborhood hotspots, and surface high‑value time windows for targeted patrols and community interventions.
- **Key findings (preview, to be updated after running):**
  - Peak incident periods cluster by **weekday and hour** (notably evening peaks for property crimes).  
  - A small set of **STARS categories** account for a large share of incidents.  
  - Incidents are **heterogeneous across neighborhoods**; resource allocation benefits from neighborhood‑level monitoring.
- **Recommendations (preview):**
  - Pilot **time‑targeted patrols** in top 2 categories during the top‑2 day‑hour windows.
  - Establish a **neighborhood weekly dashboard** to monitor changes and evaluate interventions.
  - Maintain **data lineage** awareness due to the RMS transition on 2024‑06‑03; avoid pre/post mixing without harmonization.

## Dataset Selection

- **Source (with link):** City of Cincinnati Open Data Portal — *Reported Crime (STARS Category Offenses) on or after 6/3/2024*. Portal story and metadata are available via CincyInsights and the dataset's about pages.  
  - Portal story / banner statements  
  - Associated dashboard via CincyInsights  
  - Pre‑transition records (before 2024‑06‑03) are kept in a **separate** dataset.
- **License / Terms:** Public data with anonymization and geoskewing (addresses redacted to block‑level; lat/long randomized within block). Use responsibly.
- **Rows/Columns:** ~37.9K rows, 17 columns (as of **Nov 12, 2025**). Updated daily.
- **Why this dataset:** It is timely, locally relevant, refreshed daily, and directly maps to operational decisions (staffing, patrols, community outreach).

### Columns (17)
| Column | Description | Type |
|---|---|---|
| INCIDENT_NO | Unique incident id | text |
| DateReported | Report timestamp | datetime (string in source) |
| DateFrom | Incident start | datetime (string in source) |
| DateTo | Incident end | datetime (string in source) |
| BEAT | Patrol beat | text |
| CLSD | Clearance status | text |
| STARS_Category | Offense category | text |
| DAY_OF_WEEK | Day name | text |
| CPD_Neighborhood | CPD neighborhood | text |
| Hour_From | Start hour | number |
| Hour_To | End hour | number |
| type | Part category | text |
| SNA_Neighborhood | SNA neighborhood | text |
| CC_Neighborhood | CC neighborhood | text |
| ADDRESS_X | Redacted address | text |
| LATITUDE_X | Latitude (skewed) | number |
| LONGITUDE_X | Longitude (skewed) | number |

### Stakeholder‑framed Questions
1. **When** do the highest incident levels occur (by day and hour) and do patterns differ by **STARS category**?  
2. **Where** are incidents concentrated by **neighborhood** and category?  
3. Which **categories** dominate volume post‑transition, and how stable are they **week‑over‑week**?  
4. Are there **seasonal or monthly** trends since 2024‑06‑03 (bounded by the dataset’s timeframe)?

### Visualization Plan (8+ figs + 2 interactives)
- Distribution of incidents by **weekday** (bar).  
- Distribution of incidents by **hour** (bar).  
- **Day × Hour** heatmap of counts.  
- Top **STARS categories** (bar).  
- Incidents by **neighborhood** (top‑N bar).  
- **Weekly** or **monthly** time series since 2024‑06‑03 (line).  
- **Correlation** matrix for numeric features (sanity).  
- Clearance status **CLSD** distribution (bar).  
- **Interactive 1 (Plotly):** Map incidents (scatter_geo) color by category, hover details.  
- **Interactive 2 (Plotly):** Filterable category trend (dropdown/radio) over time.

In [ ]:

# 🚀 Imports & Settings
import os, io, sys, math, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional: seaborn for quick EDA aesthetics
import seaborn as sns

# Plotly (for interactivity)
try:
    import plotly.express as px
    import plotly.graph_objects as go
    PLOTLY_AVAILABLE = True
except Exception as e:
    PLOTLY_AVAILABLE = False
    print("Plotly not available – interactive charts will be skipped. Install with `pip install plotly`.")

pd.set_option("display.max_columns", 120)
pd.set_option("display.float_format", "{:,.3f}".format)

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True
sns.set_context("notebook")


## Loading & File I/O

The full dataset should be saved to `data/Reported_STARS_Category_Offenses.csv`.  
For portability, this notebook includes a **tiny embedded sample** used only if the full CSV is not found, so the workflow runs end‑to‑end.

In [ ]:

DATA_PATH = "data/Reported_STARS_Category_Offenses.csv"

if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
    source_used = "full_csv"
else:
    print("⚠️ Full CSV not found at", DATA_PATH, "\nUsing embedded 6‑row sample for demonstration.")
    import io
    sample_io = io.StringIO("INCIDENT_NO,DateReported,DateFrom,DateTo,BEAT,CLSD,STARS_Category,DAY_OF_WEEK,CPD_Neighborhood,Hour_From,Hour_To,type,SNA_Neighborhood,CC_Neighborhood,ADDRESS_X,LATITUDE_X,LONGITUDE_X\n2024-INC-000014,2024 Jun 03 07:54:00 AM,2024 Jun 03 05:00:00 AM,2024 Jun 03 05:32:00 AM,CP3,EARLY_CLOSED,Theft from Auto,Monday,WESTWOOD,5,5,Part 1 Property,Westwood,Westwood,61XX GLENWAY AVE,39.1444,-84.6219\n2024-INC-000019,2024 Jun 03 07:13:00 AM,2024 Jun 02 07:11:00 PM,2024 Jun 02 07:20:00 PM,CP2,CLEARED_BY_ARREST_ADULT,Burglary/BE,Monday,EAST END,19,19,Part 1 Property,Linwood,East End,43XX KELLOGG AVE,39.1018,-84.4317\n2024-INC-000041,2024 Jun 03 01:08:00 PM,2024 Jun 02 12:00:00 AM,2024 Jun 03 06:00:00 AM,CP3,EARLY_CLOSED,Theft from Auto,Monday,WESTWOOD,0,6,Part 1 Property,Westwood,Westwood,1XX VIENNA WOODS DR,39.1543,-84.5868\n2024-INC-000051,2024 Jun 03 11:59:00 AM,2024 Jun 02 02:30:00 PM,2024 Jun 03 10:00:00 AM,CP3,EARLY_CLOSED,Burglary/BE,Monday,WESTWOOD,14,10,Part 1 Property,Westwood,Westwood,32XX WERK RD,39.1403,-84.6088\n2024-INC-000056,2024 Jun 03 12:28:00 PM,2024 Jun 03 06:00:00 AM,2024 Jun 03 06:30:00 AM,CP3,EARLY_CLOSED,Theft from Auto,Monday,WESTWOOD,6,6,Part 1 Property,Westwood,Westwood,23XX MONTANA AVE,39.1585,-84.5748\n2024-INC-000058,2024 Jun 03 12:57:00 PM,2024 May 31 09:00:00 AM,2024 May 31 11:35:00 AM,CP2,UNFOUNDED,Personal/Other Theft,Monday,EVANSTON,9,11,Part 1 Property,Evanston,Evanston,20XX FAIRFAX AVE,39.1362,-84.4629\n")
    df = pd.read_csv(sample_io)
    source_used = "embedded_sample"

print("Source used:", source_used)
print("Shape:", df.shape)
df.head()


## Preparation & Feature Engineering

Steps:
- **Type casting:** parse `DateReported`, `DateFrom`, `DateTo` to datetimes.
- **Time features:** derive `date`, `month`, `week`, `hour_from`, `hour_to`.
- **Standardize** neighborhood and category fields.
- **Boundaries:** The dataset begins **2024‑06‑03** due to RMS transition—avoid pre/post mixing.

In [ ]:

df = df.copy()

# Parse datetimes safely
for col in ["DateReported", "DateFrom", "DateTo"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Normalize text columns
for col in ["STARS_Category","CPD_Neighborhood","DAY_OF_WEEK","CLSD","BEAT","type"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# Derive helper time columns
if "DateReported" in df.columns:
    df["date"] = df["DateReported"].dt.date
    df["month"] = df["DateReported"].dt.to_period("M").astype(str)
    df["week"] = df["DateReported"].dt.to_period("W").astype(str)
    df["hour_reported"] = df["DateReported"].dt.hour

# Hour_From/Hour_To safety
for hcol in ["Hour_From", "Hour_To"]:
    if hcol in df.columns:
        df[hcol] = pd.to_numeric(df[hcol], errors="coerce")

# Basic health check
display(df.info())
df.describe(include="all").T.head(25)


## Analysis & Visualization

Each figure includes a short **stakeholder takeaway** focusing on actionability.

In [ ]:

if "DAY_OF_WEEK" in df.columns:
    order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    counts = df["DAY_OF_WEEK"].value_counts().reindex(order)
    ax = counts.plot(kind="bar")
    ax.set_title("Incidents by Day of Week")
    ax.set_xlabel("Day of Week"); ax.set_ylabel("Incidents")
    plt.xticks(rotation=45, ha="right")
    plt.show()
else:
    print("DAY_OF_WEEK not available.")


> **Takeaway:** Concentrations on specific weekdays can inform staffing levels and targeted community outreach windows.

In [ ]:

hour_col = None
for candidate in ["hour_reported","Hour_From"]:
    if candidate in df.columns:
        hour_col = candidate
        break

if hour_col:
    ax = df[hour_col].dropna().astype(int).plot(kind="hist", bins=24)
    ax.set_title(f"Incident Distribution by Hour ({hour_col})")
    ax.set_xlabel("Hour (0–23)"); ax.set_ylabel("Incidents")
    plt.show()
else:
    print("No hour column available for histogram.")


> **Takeaway:** Evening or late-night peaks suggest patrol realignment opportunities.

In [ ]:

if ("DAY_OF_WEEK" in df.columns) and (hour_col is not None):
    order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    pivot = (df.dropna(subset=[hour_col])
               .assign(hr=df[hour_col].astype(int))
               .groupby(["DAY_OF_WEEK","hr"])
               .size()
               .reset_index(name="n"))
    pivot["DAY_OF_WEEK"] = pd.Categorical(pivot["DAY_OF_WEEK"], categories=order, ordered=True)
    heat = pivot.pivot(index="DAY_OF_WEEK", columns="hr", values="n").fillna(0)

    sns.heatmap(heat, cmap="YlOrRd")
    plt.title("Incidents Heatmap — Day × Hour")
    plt.xlabel("Hour"); plt.ylabel("Day of Week")
    plt.show()
else:
    print("Cannot build Day × Hour heatmap.")


> **Takeaway:** High-intensity cells flag the best windows for targeted interventions (e.g., patrols, lighting, outreach).

In [ ]:

if "STARS_Category" in df.columns:
    topN = 10
    counts = df["STARS_Category"].value_counts().head(topN)
    ax = counts.plot(kind="bar")
    ax.set_title(f"Top {topN} STARS Categories")
    ax.set_xlabel("Category"); ax.set_ylabel("Incidents")
    plt.xticks(rotation=45, ha="right")
    plt.show()
else:
    print("STARS_Category not available.")


> **Takeaway:** A small subset of categories typically dominates incident volume—focus diagnostics and prevention here first.

In [ ]:

neigh_col = None
for candidate in ["CPD_Neighborhood","SNA_Neighborhood","CC_Neighborhood"]:
    if candidate in df.columns:
        neigh_col = candidate
        break

if neigh_col:
    topN = 12
    counts = df[neigh_col].value_counts().head(topN)
    ax = counts.plot(kind="bar")
    ax.set_title(f"Top {topN} Neighborhoods by Incident Count ({neigh_col})")
    ax.set_xlabel("Neighborhood"); ax.set_ylabel("Incidents")
    plt.xticks(rotation=45, ha="right")
    plt.show()
else:
    print("No neighborhood column found.")


> **Takeaway:** Prioritize monitoring and engagement in top neighborhoods; pair with specific top categories there.

In [ ]:

if "week" in df.columns:
    ts = df.groupby("week").size().reset_index(name="n").sort_values("week")
    ax = ts.set_index("week")["n"].plot(kind="line", marker="o")
    ax.set_title("Weekly Incident Counts")
    ax.set_xlabel("ISO Week"); ax.set_ylabel("Incidents")
    plt.xticks(rotation=45, ha="right")
    plt.show()
else:
    print("Week column unavailable.")


> **Takeaway:** Track week-over-week changes to evaluate whether interventions move the trend.

In [ ]:

if "CLSD" in df.columns:
    counts = df["CLSD"].value_counts()
    ax = counts.plot(kind="bar")
    ax.set_title("Clearance Status Distribution (CLSD)")
    ax.set_xlabel("Status"); ax.set_ylabel("Incidents")
    plt.xticks(rotation=45, ha="right")
    plt.show()
else:
    print("CLSD not available.")


> **Takeaway:** Clearance patterns can suggest process improvements or resource constraints.

In [ ]:

num = df.select_dtypes(include="number")
if num.shape[1] >= 2:
    corr = num.corr()
    sns.heatmap(corr, cmap="vlag", center=0)
    plt.title("Correlation Heatmap (Numeric Features)")
    plt.show()
else:
    print("Not enough numeric columns for correlation heatmap.")


> **Takeaway:** Mostly for QA here; limited numeric fields expected. Outliers or odd correlations may indicate data issues.

In [ ]:

if PLOTLY_AVAILABLE and {"LATITUDE_X","LONGITUDE_X"}.issubset(df.columns):
    sub = df.dropna(subset=["LATITUDE_X","LONGITUDE_X"]).copy()
    sub = sub.head(5000)  # guard for rendering
    fig = px.scatter_geo(
        sub,
        lat="LATITUDE_X", lon="LONGITUDE_X",
        color="STARS_Category" if "STARS_Category" in sub.columns else None,
        hover_data=[c for c in ["INCIDENT_NO","DateReported","CPD_Neighborhood","STARS_Category","CLSD"] if c in sub.columns],
        title="Incidents Map (block‑level randomized coordinates)"
    )
    fig.update_layout(height=550)
    fig.show()
else:
    print("Plotly unavailable or missing lat/lon; skipping map.")


> **Takeaway:** Spatial clustering (even with block‑level jitter) helps prioritize area‑based responses and environmental design changes.

In [ ]:

if PLOTLY_AVAILABLE and {"STARS_Category","month"}.issubset(df.columns):
    cats = df["STARS_Category"].dropna().value_counts().head(8).index.tolist()
    data_by_cat = (
        df[df["STARS_Category"].isin(cats)]
        .groupby(["STARS_Category","month"]).size().reset_index(name="n")
        .sort_values(["STARS_Category","month"])
    )
    # Build a single figure with dropdown to switch category
    base_cat = cats[0] if cats else None
    if base_cat:
        base = data_by_cat[data_by_cat["STARS_Category"]==base_cat]
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=base["month"], y=base["n"], mode="lines+markers", name=base_cat))
        buttons = []
        for c in cats:
            dat = data_by_cat[data_by_cat["STARS_Category"]==c]
            buttons.append(dict(
                method="update",
                label=c,
                args=[{"x":[dat["month"]], "y":[dat["n"]], "type":"scatter"},
                      {"title": f"Monthly Incidents — {c}"}]
            ))
        fig.update_layout(
            title=f"Monthly Incidents — {base_cat}",
            updatemenus=[dict(type="dropdown", x=0.0, y=1.15, showactive=True, buttons=buttons)],
            xaxis_title="Month", yaxis_title="Incidents", height=500
        )
        fig.show()
else:
    print("Plotly unavailable or insufficient columns for category trend.")


> **Takeaway:** Track top categories month‑over‑month for capacity planning and campaign timing.

## Conclusions
- **Peak windows** (by day/hour) reveal when incremental staffing could yield the largest marginal benefit.  
- **Top categories** dominate volume; prevention and outreach should start there.  
- **Neighborhood hotspots** show where multi‑stakeholder engagement (CPD + neighborhood councils) can concentrate efforts.  
- **Trends** (weekly/monthly) support continuous evaluation—establish KPIs, alerting, and dashboards.

**Limitations**
- Geolocation is **block‑level randomized**; do not interpret as exact locations.  
- The dataset excludes **calls for service, arrests, and outcomes**; consider integrating with complementary sources.  
- RMS transition on **2024‑06‑03** means schema differs from earlier data. Avoid pre/post joins without harmonization.

## Appendix & References
- City of Cincinnati Open Data (Public Safety) — Reported Crime (STARS Category Offenses) on or after 6/3/2024  
- CincyInsights dashboards and banner statements for data caveats  
- Data usage & how‑to guide on the portal

**Repro Notes**
- Expected file path: `data/Reported_STARS_Category_Offenses.csv`  
- If missing, this notebook runs with an embedded 6‑row sample for demonstration.